In [9]:
from google.colab import files
uploaded = files.upload()
!mkdir -p /content/data
!mv dataset.csv /content/data/parking.csv


Saving dataset.csv to dataset.csv


In [11]:
# Dynamic Parking Pricing - Final Colab Notebook

#Setup
!pip install pathway -q
!pip install haversine -q

import pandas as pd
import numpy as np
import os

#Load Data
os.makedirs('/content/data', exist_ok=True)
from google.colab import files
uploaded = files.upload()
!mv dataset.csv /content/data/parking.csv


#Load & format
raw = pd.read_csv('/content/data/parking.csv')
raw.columns = raw.columns.str.strip()
raw['timestamp'] = pd.to_datetime(raw['LastUpdatedDate'] + ' ' + raw['LastUpdatedTime'], dayfirst=True)
raw['day'] = raw['timestamp'].dt.date

# Train split
latest_days = sorted(raw['day'].unique())[-10:]
df_train = raw[~raw['day'].isin(latest_days)].copy()

# Feature Engineering
df_train['occ_ratio'] = df_train['Occupancy'] / df_train['Capacity']
df_train['queue_intensity'] = df_train['QueueLength'] / df_train['Capacity']
df_train['TrafficConditionNearby'] = df_train['TrafficConditionNearby'].map({'low': 0, 'medium': 1, 'high': 2})
df_train['VehicleType'] = df_train['VehicleType'].astype('category').cat.codes

# ✅ STEP 2: Baseline Model
class LinearPricer:
    def __init__(self, alpha=0.3, base=10):
        self.alpha = alpha
        self.price = base

    def update(self, occ, cap):
        self.price += self.alpha * (occ / cap)
        return round(max(5, min(self.price, 20)), 2)

baseline = LinearPricer()
baseline_prices = []
for _, row in df_train.iterrows():
    baseline_prices.append(baseline.update(row['Occupancy'], row['Capacity']))
df_train['baseline_price'] = baseline_prices

#Demand-Based Model
from sklearn.preprocessing import MinMaxScaler

class DemandPricer:
    def __init__(self, lamb=0.8):
        self.scaler = MinMaxScaler()
        self.lamb = lamb

    def fit_scaler(self, df):
        features = ['occ_ratio', 'queue_intensity', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']
        self.scaler.fit(df[features])

    def predict(self, row):
        x = self.scaler.transform(pd.DataFrame([row]))[0]
        demand = np.dot(x, [0.4, 0.3, -0.1, 0.1, 0.1])
        price = 10 * (1 + self.lamb * demand)
        return round(np.clip(price, 5, 20), 2)

demand_model = DemandPricer()
demand_model.fit_scaler(df_train)

demand_prices = []
for _, row in df_train.iterrows():
    input_row = row[['occ_ratio', 'queue_intensity', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']]
    demand_prices.append(demand_model.predict(input_row))
df_train['demand_price'] = demand_prices

# Competitive Model
from haversine import haversine, Unit

class CompetitivePricer:
    def __init__(self, lots_info, lamb=0.8):
        self.base_model = DemandPricer(lamb)
        self.lots_info = lots_info

    def fit_scaler(self, df):
        self.base_model.fit_scaler(df)

    def predict(self, row):
        demand_price = self.base_model.predict(row[['occ_ratio', 'queue_intensity', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']])
        lat1, lon1 = row['Latitude'], row['Longitude']
        lot_id = row['SystemCodeNumber']
        nearby_prices = []

        for _, other in self.lots_info.iterrows():
            if other['SystemCodeNumber'] == lot_id:
                continue
            dist = haversine((lat1, lon1), (other['Latitude'], other['Longitude']), unit=Unit.METERS)
            if dist <= 300:
                nearby_prices.append(other['CurrentPrice'])

        final_price = demand_price
        occ_ratio = row['Occupancy'] / row['Capacity']
        for p in nearby_prices:
            if occ_ratio > 0.95 and p + 1 < demand_price:
                final_price -= 0.5
            elif occ_ratio < 0.7 and p - 1 > demand_price:
                final_price += 0.5

        return round(np.clip(final_price, 5, 20), 2)

lots_info = pd.DataFrame([
    {'SystemCodeNumber': 'FAKE1', 'Latitude': df_train.iloc[0]['Latitude'] + 0.001, 'Longitude': df_train.iloc[0]['Longitude'] + 0.001, 'CurrentPrice': 10},
    {'SystemCodeNumber': 'FAKE2', 'Latitude': df_train.iloc[0]['Latitude'] - 0.001, 'Longitude': df_train.iloc[0]['Longitude'] - 0.001, 'CurrentPrice': 12},
])

comp_model = CompetitivePricer(lots_info)
comp_model.fit_scaler(df_train)

comp_prices = []
for _, row in df_train.iterrows():
    comp_prices.append(comp_model.predict(row))
df_train['competitive_price'] = comp_prices

#Save all predictions
output_path = '/content/output_prices.csv'
df_train[['SystemCodeNumber', 'timestamp', 'Occupancy', 'Capacity', 'baseline_price', 'demand_price', 'competitive_price']].to_csv(output_path, index=False)
print(f"Saved final output to: {output_path}")


Saving dataset.csv to dataset.csv
Saved final output to: /content/output_prices.csv
